```{contents}
```
## Multi-Tenant Execution

**Multi-tenant execution** in LangGraph refers to the ability to **safely and efficiently run multiple independent users, teams, or organizations on the same LangGraph infrastructure**, while preserving **data isolation, performance fairness, security boundaries, and operational control**.

This is essential for **enterprise platforms, SaaS AI products, and internal company AI systems**.

---

### **1. Core Intuition**

Without multi-tenancy, every user would require:

* A dedicated LangGraph runtime
* Separate memory, state, and infrastructure
* Complex duplication of resources

With **multi-tenancy**, a single LangGraph platform hosts **many independent logical systems**, each called a **tenant**.

> **One infrastructure → many isolated AI systems**

---

### **2. What is a Tenant?**

A **tenant** is a logically isolated execution domain with:

| Layer    | Isolation                   |
| -------- | --------------------------- |
| State    | Separate state store        |
| Memory   | Separate vector memory      |
| Graphs   | Tenant-specific graphs      |
| Agents   | Tenant-specific agents      |
| Tools    | Tenant-specific permissions |
| Models   | Tenant-specific routing     |
| Logs     | Separate audit trails       |
| Security | Independent auth policies   |

---

### **3. Execution Architecture**

```
Clients (Tenant A, B, C)
        │
   API Gateway + Auth
        │
 Tenant Resolver
        │
 LangGraph Orchestrator
        │
 ┌──────────┬──────────┬──────────┐
 │Tenant A  │Tenant B  │Tenant C  │
 │Runtime   │Runtime   │Runtime   │
 │State     │State     │State     │
 │Memory    │Memory    │Memory    │
 └──────────┴──────────┴──────────┘
```

---

### **4. Tenant Identification & Routing**

Every request carries a **tenant identifier**:

```python
def resolve_tenant(request):
    return request.headers["X-Tenant-ID"]
```

The LangGraph runtime injects the tenant ID into the **execution context**:

```python
graph.invoke(input, config={
    "configurable": {
        "tenant_id": tenant_id
    }
})
```

---

### **5. Tenant-Isolated State & Memory**

**State keys** are automatically namespaced:

```text
tenantA:thread123:state
tenantB:thread123:state
```

Memory stores use **tenant partitions**:

```python
vector_store = Pinecone(index=f"{tenant_id}_memory")
```

---

### **6. Graph Isolation**

Tenants may:

* Share the same graph definition
* Or run **custom graph versions**

```python
graphs = {
    "tenantA": sales_graph,
    "tenantB": legal_graph,
    "tenantC": research_graph
}
```

---

### **7. Security & Access Control**

| Layer  | Mechanism                 |
| ------ | ------------------------- |
| API    | OAuth / JWT               |
| Tenant | Tenant-scoped permissions |
| Graph  | Role-based execution      |
| Tools  | Per-tenant allowlist      |
| Data   | Encrypted per tenant      |
| Logs   | Separate audit streams    |

---

### **8. Performance & Fairness Controls**

| Mechanism     | Purpose                 |
| ------------- | ----------------------- |
| Rate limiting | Prevent noisy neighbors |
| Quotas        | Enforce usage plans     |
| Worker pools  | Fair scheduling         |
| Resource caps | Prevent overload        |
| Cost tracking | Per-tenant billing      |

---

### **9. Deployment Model**

```
Kubernetes Cluster
    |
LangGraph Runtime Pods
    |
Tenant-Aware Load Balancer
    |
Tenant-Isolated Databases & Memory
```

---

### **10. Minimal Multi-Tenant Example**

```python
def invoke_for_tenant(tenant_id, input_data):
    return graph.invoke(
        input_data,
        config={"configurable": {"tenant_id": tenant_id}}
    )
```

---

### **11. Production Benefits**

| Benefit       | Result                |
| ------------- | --------------------- |
| Scalability   | Thousands of tenants  |
| Security      | Strong isolation      |
| Efficiency    | Shared infrastructure |
| Cost control  | Metered usage         |
| Customization | Per-tenant AI systems |

---

### **12. Mental Model**

LangGraph multi-tenancy behaves like:

> **Kubernetes namespaces for intelligent workflows**

Each tenant lives in a secure, isolated logical universe—
while sharing the same powerful orchestration engine.


### Demonstration

In [1]:
"""
One-cell minimal demonstration of Multi-Tenant execution in LangGraph
Works conceptually and mirrors real production behavior.
"""

from typing import TypedDict
from langgraph.graph import StateGraph, END

# -------------------------
# 1. Define shared state
# -------------------------
class State(TypedDict):
    tenant_id: str
    message: str
    response: str

# -------------------------
# 2. Tenant-aware node
# -------------------------
def tenant_node(state: State):
    tenant = state["tenant_id"]
    msg = state["message"]
    return {"response": f"[{tenant}] processed: {msg}"}

# -------------------------
# 3. Build graph
# -------------------------
builder = StateGraph(State)
builder.add_node("processor", tenant_node)
builder.set_entry_point("processor")
builder.add_edge("processor", END)
graph = builder.compile()

# -------------------------
# 4. Multi-tenant execution
# -------------------------
def invoke_for_tenant(tenant_id, message):
    return graph.invoke({
        "tenant_id": tenant_id,
        "message": message
    })

# -------------------------
# 5. Simulated tenants
# -------------------------
print(invoke_for_tenant("Tenant-A", "Generate sales report"))
print(invoke_for_tenant("Tenant-B", "Analyze legal contract"))
print(invoke_for_tenant("Tenant-C", "Summarize research paper"))


{'tenant_id': 'Tenant-A', 'message': 'Generate sales report', 'response': '[Tenant-A] processed: Generate sales report'}
{'tenant_id': 'Tenant-B', 'message': 'Analyze legal contract', 'response': '[Tenant-B] processed: Analyze legal contract'}
{'tenant_id': 'Tenant-C', 'message': 'Summarize research paper', 'response': '[Tenant-C] processed: Summarize research paper'}
